Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

- date – дата совершения события

- event - событие

--  app_install – установка приложения

-- app_start – открыть приложения

-- registration – зарегистрироваться 

-- search – перейти на страницу поиска товаров (каталог)

-- open_item – открыть товар

-- choose_item – отправить товар в корзину

-- tap_basket – перейти в корзину

-- purchase – подтверждение покупки

- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь

-- yandex-direct – Яндекс директ

-- google_ads – реклама в Google

-- vk_ads – реклама в ВК

-- instagram_ads – реклама в instagram

-- facebook_ads – реклама в facebook

-- referral – акция «приведи друга»

Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [1]:
import pandas as pd
import numpy as np

Читаем данные, смотрим их характеристики:

In [2]:
df = pd.read_csv('/home/jupyter-k.bataeva-14/Product_Analytics/KC_case_data .csv')
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [3]:
df.shape


(2747968, 8)

In [4]:
df.dtypes

date             object
event            object
purchase_sum    float64
os_name          object
device_id         int64
gender           object
city             object
utm_source       object
dtype: object

In [5]:
df.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [6]:
df.date = pd.to_datetime(df.date) #переводим колонку в формат даты
df.date.head()

0   2020-01-01
1   2020-01-01
2   2020-01-01
3   2020-01-01
4   2020-01-01
Name: date, dtype: datetime64[ns]

MAU февраля:

In [7]:
df.date.dt.month

0          1
1          1
2          1
3          1
4          1
          ..
2747963    3
2747964    3
2747965    3
2747966    3
2747967    3
Name: date, Length: 2747968, dtype: int64

In [8]:
df.query('date.dt.month == 2').device_id.nunique()

75032

Количество установок в январе:

In [9]:
df.query('date.dt.month == 1 & event== "app_install"').event.count()

80297

Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? 

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [10]:
id_list_inst = df.query('event == "app_install"')['device_id'].to_list() #отбираю только айди с установкой

In [11]:
df_cog = df[df['device_id'].isin(id_list_inst)].query('event == "app_install" or event == "purchase"') #получаю дф с когортами
df_cog = df_cog.sort_values(by = ['date', 'event', 'device_id']) #сортирую по полям, чтобы было удобнее
df_cog

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
2515778,2020-01-01,app_install,NaN,ios,4378,male,Saint-Petersburg,google_ads
2515286,2020-01-01,app_install,NaN,android,4505,female,Moscow,yandex-direct
2515922,2020-01-01,app_install,NaN,ios,4520,female,Saint-Petersburg,instagram_ads
2517537,2020-01-01,app_install,NaN,android,5248,female,Saint-Petersburg,facebook_ads
2516068,2020-01-01,app_install,NaN,ios,5524,male,Saint-Petersburg,vk_ads
...,...,...,...,...,...,...,...,...
2507067,2020-03-31,purchase,141.0,android,34964615,female,Saint-Petersburg,instagram_ads
2506627,2020-03-31,purchase,613.0,ios,35057676,male,Moscow,instagram_ads
2506590,2020-03-31,purchase,257.5,android,35171283,female,Moscow,vk_ads
2507015,2020-03-31,purchase,351.5,android,35315632,female,Moscow,-


In [12]:
df_dropped1 = df_cog.drop_duplicates(subset = ['event','device_id'], keep = 'first') #оставляю только первую покупку для кажого айди

In [14]:
#добавляю колонку с датой регистрации
reg_date = df_dropped1.query('event == "app_install"')[['date', 'device_id']].rename(columns = {'date':'reg_date'}) 

In [15]:
df_dropped1 =df_dropped1.merge(reg_date , how= 'left', on='device_id')

In [17]:
#добавляю колонку с датой покупки
purch_date = df_dropped1.query('event == "purchase"')[['date', 'device_id']].rename(columns = {'date':'purch_date'})

In [18]:
df_dropped1 =df_dropped1.merge(purch_date , how= 'left', on='device_id')

In [19]:
#считаю количество установок по когортам
inst_count = df_dropped1.query('event == "app_install"').groupby('date').device_id.count()

In [20]:
#дропаю дубликаты по айди, чтобы создать колонку с фактом покупки по когортам
df_dropped2 = df_dropped1[['date', 'device_id','reg_date', 'purch_date']].drop_duplicates('device_id', keep = 'first')

In [21]:
#создаю колонку: конвертировался ли пользователь за 7 дней?
df_dropped2['conv_purch'] = np.where((df_dropped2.reg_date - df_dropped2.purch_date).dt.days <=7, 1 ,0)

In [22]:
#считаю количество пользователей с покупкой в первые 7 дней по когортам
conv_count = df_dropped2.groupby('date').conv_purch.sum()
conv_count

date
2020-01-01    2308
2020-01-02    1977
2020-01-03    1400
2020-01-04    1017
2020-01-05     910
              ... 
2020-03-27     277
2020-03-28     209
2020-03-29     223
2020-03-30     204
2020-03-31     241
Name: conv_purch, Length: 91, dtype: int64

In [23]:
#считаю конверсию
(conv_count/ inst_count).idxmax()

Timestamp('2020-01-01 00:00:00')

In [24]:
((conv_count/ inst_count).max()*100).round(1)

64.5

### Вывод
когорта с максимальной конверсией - 2020-01-01, 64.5%

С какого платного маркетингового канала пришло больше всего новых пользователей? 

Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

In [37]:
df[df['device_id'].isin(id_list_inst)].groupby('event').device_id.count().sort_values()

event
register        63526
purchase       115225
app_install    154597
tap_basket     307130
choose_item    437804
search         576040
app_start      608701
Name: device_id, dtype: int64

In [38]:
df[~df['device_id'].isin(id_list_inst)].groupby('event').device_id.count().sort_values()

event
register        14784
purchase        26158
tap_basket      70535
choose_item    100865
search         132599
app_start      140004
Name: device_id, dtype: int64

In [26]:
df.query('event== "app_install"').groupby('utm_source').device_id.count().sort_values(ascending = False)

utm_source
-                32460
yandex-direct    29368
google_ads       26286
vk_ads           23189
instagram_ads    20096
facebook_ads     13916
referal           9282
Name: device_id, dtype: int64

Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?
- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [27]:
df.query('event== "app_start"').groupby('utm_source').device_id.nunique() #кол-во юзеров, впервые открывших приложение 

utm_source
-                52268
facebook_ads     25957
google_ads       38092
instagram_ads    31045
referal          15924
vk_ads           34079
yandex-direct    40707
Name: device_id, dtype: int64

In [28]:
df.query('event== "purchase"').groupby('utm_source').device_id.nunique() #кол-во юзеров, впервые совершивших покупку

utm_source
-                16598
facebook_ads      9017
google_ads       11339
instagram_ads    10762
referal           6362
vk_ads           12364
yandex-direct    12028
Name: device_id, dtype: int64

In [29]:
CR = df.query('event== "purchase"').groupby('utm_source').device_id.nunique() / df.query('event== "app_start"').groupby('utm_source').device_id.nunique()
CR.sort_values(ascending = True)

utm_source
yandex-direct    0.295477
google_ads       0.297674
-                0.317556
instagram_ads    0.346658
facebook_ads     0.347382
vk_ads           0.362804
referal          0.399523
Name: device_id, dtype: float64

Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [30]:
df.query('purchase_sum> 0').drop_duplicates(subset = ['event','device_id'], keep = 'first').groupby('utm_source').purchase_sum.median().sort_values(ascending = False)

utm_source
-                398.5
referal          395.5
instagram_ads    393.5
vk_ads           393.0
yandex-direct    392.5
google_ads       390.5
facebook_ads     389.0
Name: purchase_sum, dtype: float64

Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Какой платный канал привлечения имеет самый высокий ROMI? 


In [31]:
ads_costs = pd.Series({'facebook_ads': '8590498', 
                       'google_ads': '10534878', 
                       'instagram_ads': '8561626', 
                       'vk_ads': '9553531',
                        'yandex-direct': '10491707'},
                     index=['facebook_ads', 'google_ads', 'instagram_ads', 'vk_ads', 'yandex-direct']) #создаю серию с рекламой
ads_costs = ads_costs.astype(float)

In [32]:
revenue_by_cahnnel = df.groupby('utm_source').purchase_sum.sum() #считаем прибыль с каждого канала
revenue_by_cahnnel.drop(['referal', '-'], axis = 0) #выкидываем ненужные строки

utm_source
facebook_ads     12249901.0
google_ads       12868276.0
instagram_ads    14546969.0
vk_ads           16389652.5
yandex-direct    13915368.0
Name: purchase_sum, dtype: float64

In [33]:
ROMI = (revenue_by_cahnnel- ads_costs) / ads_costs
ROMI

-                     NaN
facebook_ads     0.425983
google_ads       0.221493
instagram_ads    0.699090
referal               NaN
vk_ads           0.715560
yandex-direct    0.326321
dtype: float64